In [ ]:
import os
import numpy as np
import librosa
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

data_dir = r"/content/drive/MyDrive/new"

def extract_features(file_path):
    try:
        audio, sr = librosa.load(file_path, duration=3, offset=0.5)
        mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13)
        return np.mean(mfccs.T, axis=0)
    except Exception as e:
        print(f"Error loading file {file_path}: {e}")
        return None

features = []
labels = []
for folder_name in os.listdir(data_dir):
    emotion = folder_name.split('-')[-1].lower()
    folder_path = os.path.join(data_dir, folder_name)

    if os.path.isdir(folder_path): # Added this check to ensure it's a directory
        for file_name in os.listdir(folder_path):
            if file_name.endswith('.wav'):
                file_path = os.path.join(folder_path, file_name)
                mfcc = extract_features(file_path)
                if mfcc is not None:
                    features.append(mfcc)
                    labels.append(emotion)

# Moved model training outside the loop
x = np.array(features)
y = np.array(labels)

# Check if there are enough samples to split
if len(x) > 1:
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
    model = RandomForestClassifier()
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    print("accuracy", accuracy_score(y_test, y_pred))
    print("classification_report", classification_report(y_test, y_pred))
else:
    print("Not enough samples to train the model.")

accuracy 0.94375
classification_report                        precision    recall  f1-score   support

             oaf_fear       0.91      0.96      0.93        45
            oaf_happy       0.89      0.91      0.90        35
          oaf_neutral       0.97      1.00      0.99        37
oaf_pleasant_surprise       1.00      0.91      0.95        43

             accuracy                           0.94       160
            macro avg       0.94      0.94      0.94       160
         weighted avg       0.95      0.94      0.94       160



In [ ]:
from google.colab import files
uploaded=files.upload()
def predict_emotion(file_path):
  mfcc=extract_features(file_path)
  if mfcc is not None:
    return model.predict(mfcc.reshape(1, -1))[0]
  else:
    return "counld not process audio"
for fname in uploaded.keys():
        print(f"{fname}: {predict_emotion(fname)}")

Saving OAF_bought_happy.wav to OAF_bought_happy.wav
OAF_bought_happy.wav: oaf_happy
